# Import

In [1]:
import torch
import torchvision

from glcic.networks.completion_network import CompletionNetwork
from glcic.networks.discriminators import Discriminator
from glcic.trainers.conjugate_trainer import train
from glcic.utils import *

# Prepare

In [2]:
# load the dataset
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = get_dataloader(train_dataset_dir, train_dataset_dir+"barn/00001008.jpg", batch_size=2)

In [3]:
# others
cn = CompletionNetwork().cuda()
cn.load('../logs/models/cn_scrapped_weights')
cn_optimizer = torch.optim.Adadelta(cn.parameters())

discriminator = Discriminator().cuda()
discriminator_optimizer = torch.optim.Adadelta(discriminator.parameters())
_ = load_checkpoint("../logs/phase2_checkpoints/",discriminator, discriminator_optimizer)
discriminator_optimizer = torch.optim.Adadelta(discriminator.parameters()) # reset

num_batch = 10
replacement_val = torch.tensor([0.5062325495504219, 0.4255871700324652, 0.38299278586700136])

Load: load_state dict from ../logs/models/cn_scrapped_weights


# Test

In [4]:
# conjugate training
train(
    cn,
    discriminator,
    cn_optimizer,
    discriminator_optimizer,
    train_dataloader,
    num_batch,
    replacement_val,
    )


### BATCH 1/10 ###
mse grad norm:  46285.921875 , grad norm:  1.135998249053955
alpha:  407.447519387262
loss: 955.1412353515625 for the completion loss, 3.903121978510171e-05 for the discriminator loss

### BATCH 2/10 ###
mse grad norm:  604274.375 , grad norm:  1823705.25
alpha:  403.3763576365842
loss: 15342.25390625 for the completion loss, 323.8081359863281 for the discriminator loss

### BATCH 3/10 ###
mse grad norm:  80955.390625 , grad norm:  3.054342284770669e-18
alpha:  2.650501583553806e+20
loss: 23594.56640625 for the completion loss, 7785.72802734375 for the discriminator loss

### BATCH 4/10 ###
mse grad norm:  59347.2734375 , grad norm:  inf
alpha:  2.6239965677182678e+20
loss: 3.3484043679017803e+21 for the completion loss, 2.7234975499818023e+21 for the discriminator loss

### BATCH 5/10 ###
mse grad norm:  26782.689453125 , grad norm:  0.0


ZeroDivisionError: float division by zero